In [22]:
!pip install xarray
!pip install rasterio
!pip install geopandas

   ---------------------------------------- 0.0/16.2 MB ? eta -:--:--
   ------- -------------------------------- 3.1/16.2 MB 14.2 MB/s eta 0:00:01
   ----------------- ---------------------- 7.1/16.2 MB 16.8 MB/s eta 0:00:01
   -------------------------- ------------- 10.7/16.2 MB 17.2 MB/s eta 0:00:01
   ----------------------------------- ---- 14.2/16.2 MB 17.5 MB/s eta 0:00:01
   ---------------------------------------- 16.2/16.2 MB 17.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ----------------------- ---------------- 3.7/6.3 MB 18.1 MB/s eta 0:00:01
   ---------------------------------------- 6.3/6.3 MB 17.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 15.1 MB/s eta 0:00:00


In [24]:
import requests
import xarray as xr
import rasterio
import pandas as pd
import geopandas as gpd
from io import BytesIO

# === 1. Download NetCDF from WCS ===

def download_netCDF():
    url = "https://geo.weather.gc.ca/geomet-climate"
    params = {
        "SERVICE": "WCS",
        "VERSION": "2.0.1",
        "REQUEST": "GetCoverage",
        "COVERAGEID": "DCS.TM.RCP85.YEAR.2081-2100_PCTL50",
        "SUBSETTINGCRS": "EPSG:4326",
        "SUBSET": ["x(-82,-76)", "y(42,45)"],
        "FORMAT": "image/netcdf"
    }
    response = requests.get(url, params=params)
    with open("temperature_projection.nc", "wb") as f:
        f.write(response.content)
    print("✅ Downloaded NetCDF data.")

    # Open and read NetCDF
    ds = xr.open_dataset("temperature_projection.nc")
    print(ds)
    ds.to_dataframe().to_csv("temperature_projection.csv")
    print("✅ Converted NetCDF to CSV.")

# === 2. Download GeoTIFF from WCS ===

def download_geotiff():
    url = "https://geo.weather.gc.ca/geomet-climate"
    params = {
        "SERVICE": "WCS",
        "VERSION": "2.0.1",
        "REQUEST": "GetCoverage",
        "COVERAGEID": "CMIP5.PR.RCP85.YEAR.ANO_PCTL50",
        "RANGESUBSET": "B2060:B2070",
        "SUBSETTINGCRS": "EPSG:4326",
        "SUBSET": [
            "x(-99.525146484375,-95.130615234375)",
            "y(48.21003212234042,51.055207338584964)"
        ],
        "FORMAT": "image/tiff"
    }
    response = requests.get(url, params=params)
    with open("precipitation_projection.tif", "wb") as f:
        f.write(response.content)
    print("✅ Downloaded GeoTIFF data.")

    # Read with rasterio
    with rasterio.open("precipitation_projection.tif") as src:
        print(f"✅ GeoTIFF CRS: {src.crs}")
        print(f"✅ GeoTIFF Bounds: {src.bounds}")
        print(f"✅ GeoTIFF Shape: {src.shape}")

# === 3. Query WFS climate feature data ===

def query_wfs_by_station(station_id="1053"):
    url = f"https://geo.weather.gc.ca/geomet/features/collections/climate-daily/items"
    params = {"STN_ID": station_id}
    response = requests.get(url, params=params)
    data = response.json()
    df = pd.json_normalize(data["features"])
    df.to_csv("climate_station_data.csv", index=False)
    print(f"✅ Saved station {station_id} climate data to CSV.")

# === Run All ===

if __name__ == "__main__":
    download_netCDF()
    download_geotiff()
    query_wfs_by_station("1053")  # You can change this to another station ID


✅ Downloaded NetCDF data.
<xarray.Dataset> Size: 11kB
Dimensions:  (lat: 36, lon: 72)
Coordinates:
  * lat      (lat) float64 288B 42.04 42.12 42.21 42.29 ... 44.79 44.88 44.96
  * lon      (lon) float64 576B -81.96 -81.88 -81.79 ... -76.21 -76.13 -76.04
Data variables:
    Band1    (lat, lon) float32 10kB ...
Attributes:
    GDAL_TIFFTAG_RESOLUTIONUNIT:  2
    GDAL_TIFFTAG_XRESOLUTION:     72.0
    GDAL_TIFFTAG_YRESOLUTION:     72.0
    Conventions:                  CF-1.5
    GDAL:                         GDAL 3.5.2, released 2022/09/02
    history:                      Tue Mar 25 15:06:46 2025: GDAL CreateCopy( ...
✅ Converted NetCDF to CSV.
✅ Downloaded GeoTIFF data.
✅ GeoTIFF CRS: None
✅ GeoTIFF Bounds: BoundingBox(left=-99.525146484375, bottom=48.21003212234043, right=-95.130615234375, top=51.055207338584964)
✅ GeoTIFF Shape: (3, 4)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)